<a href="https://colab.research.google.com/github/viniciusrpb/cic0193_machinelearning/blob/main/cap9_3_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
from keras.models import Sequential
from keras.layers import Dense,MaxPooling2D,Activation,Flatten,Conv2D,BatchNormalization,Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from sklearn.metrics import classification_report
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp -r "/content/drive/My Drive/leafs" "leafs"

In [56]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range =20,
                                   fill_mode='nearest',
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255.)

train_set = train_datagen.flow_from_directory("leafs/train",
                                              subset='training',
                                              target_size=(224,224),
                                              class_mode="categorical",
                                              batch_size=32)

validation_set = train_datagen.flow_from_directory("leafs/train",
                                              subset='validation',
                                              target_size=(224,224),
                                              class_mode="categorical",
                                              batch_size=32)

test_set = test_datagen.flow_from_directory("leafs/test",
                                            target_size=(224,224),
                                            class_mode="categorical",
                                            batch_size=32)



Found 160 images belonging to 4 classes.
Found 40 images belonging to 4 classes.
Found 100 images belonging to 4 classes.


In [43]:
kernel_dimensions = (5,5)
tam_strides = (1,1)
number_filters = 50
number_of_neurons = 30

num_classes = 4

In [55]:
model = Sequential()

model.add(Conv2D(filters=number_filters,padding="same",kernel_size=kernel_dimensions,input_shape=(224,224,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(filters=25,padding="same",kernel_size=kernel_dimensions))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(filters=15,padding="same",kernel_size=kernel_dimensions))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(number_of_neurons))
model.add(Activation("relu"))
#model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation("softmax"))

In [49]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 64, 64, 50)        3800      
_________________________________________________________________
activation_30 (Activation)   (None, 64, 64, 50)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 32, 32, 50)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 25)        31275     
_________________________________________________________________
activation_31 (Activation)   (None, 32, 32, 25)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 16, 16, 25)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 16, 16, 15)      

In [50]:
sgd = SGD(learning_rate=0.1)

model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [51]:
model.fit(train_set,validation_data=validation_set,epochs=10)

Epoch 1/10
5/5 [==============================] - 15s 3s/step - loss: 1.3969 - accuracy: 0.2313 - val_loss: 1.3839 - val_accuracy: 0.3000
Epoch 2/10
5/5 [==============================] - 13s 3s/step - loss: 1.3905 - accuracy: 0.2438 - val_loss: 1.3851 - val_accuracy: 0.2750
Epoch 3/10
5/5 [==============================] - 13s 3s/step - loss: 1.3934 - accuracy: 0.1937 - val_loss: 1.3854 - val_accuracy: 0.2250
Epoch 4/10
5/5 [==============================] - 13s 3s/step - loss: 1.3896 - accuracy: 0.2188 - val_loss: 1.3831 - val_accuracy: 0.2500
Epoch 5/10
5/5 [==============================] - 13s 3s/step - loss: 1.3840 - accuracy: 0.2750 - val_loss: 1.3816 - val_accuracy: 0.3000
Epoch 6/10
5/5 [==============================] - 13s 3s/step - loss: 1.3832 - accuracy: 0.2875 - val_loss: 1.3790 - val_accuracy: 0.3000
Epoch 7/10
5/5 [==============================] - 12s 3s/step - loss: 1.3811 - accuracy: 0.2750 - val_loss: 1.3795 - val_accuracy: 0.2500
Epoch 8/10
5/5 [==================

In [36]:
y_prob = model.predict(test_set)
y_pred = np.argmax(y_prob,axis=1)

print(classification_report(test_set.classes,y_pred))

              precision    recall  f1-score   support

           0       0.25      1.00      0.40        25
           1       0.00      0.00      0.00        25
           2       0.00      0.00      0.00        25
           3       0.00      0.00      0.00        25

    accuracy                           0.25       100
   macro avg       0.06      0.25      0.10       100
weighted avg       0.06      0.25      0.10       100



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Transfer Learning

In [73]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape=(224,224,3),
                                include_top=False,
                                weights='imagenet')



In [74]:
for layer in pre_trained_model.layers:
  layer.trainable=False

In [75]:
ans = layers.Flatten()(pre_trained_model.output)

ans = layers.Dense(1024,activation="relu")(ans)

ans = layers.Dropout(0.2)(ans)

ans = layers.Dense(num_classes,activation="softmax")(ans)

model_tl = Model(pre_trained_model.input,ans)

In [66]:
#model_tl.summary()

In [76]:
model_tl.compile(optimizer=SGD(learning_rate=0.1),
                 loss = "categorical_crossentropy",
                 metrics=["accuracy"])

In [77]:
history_fine = model_tl.fit(train_set,
                            epochs=20,
                            validation_data=validation_set
                            #initial_epoch=history.epoch[-1],
                            )

Epoch 1/20
5/5 [==============================] - 23s 4s/step - loss: 675.9012 - accuracy: 0.2625 - val_loss: 25.8998 - val_accuracy: 0.2500
Epoch 2/20
5/5 [==============================] - 15s 3s/step - loss: 65.5334 - accuracy: 0.2875 - val_loss: 1.3992 - val_accuracy: 0.3250
Epoch 3/20
5/5 [==============================] - 15s 3s/step - loss: 2.3431 - accuracy: 0.3125 - val_loss: 3.6120 - val_accuracy: 0.2500
Epoch 4/20
5/5 [==============================] - 15s 3s/step - loss: 8.0415 - accuracy: 0.2688 - val_loss: 1.3680 - val_accuracy: 0.2750
Epoch 5/20
5/5 [==============================] - 15s 3s/step - loss: 1.6780 - accuracy: 0.3000 - val_loss: 1.3273 - val_accuracy: 0.4250
Epoch 6/20
5/5 [==============================] - 15s 3s/step - loss: 2.1062 - accuracy: 0.3000 - val_loss: 1.3790 - val_accuracy: 0.2750
Epoch 7/20
5/5 [==============================] - 15s 3s/step - loss: 1.4339 - accuracy: 0.2438 - val_loss: 1.3864 - val_accuracy: 0.2500
Epoch 8/20
5/5 [==============